# Dynamically route logic based on input

This notebook covers how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:

    Using a RunnableBranch.
    Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

We’ll illustrate both methods using a two step sequence where the first step classifies an input question as being about LangChain, Anthropic, or Other, then routes to a corresponding prompt chain.

# Using a RunnableBranch

A RunnableBranch is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it’s invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Here’s an example of what it looks like in action:

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, module_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
# from langchain.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

First, let’s create a chain that will identify incoming questions as being about LangChain, Anthropic, or Other:

In [4]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

In [5]:
chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

Now, let’s create three sub chains:

In [6]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

In [7]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [8]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [9]:
full_chain.invoke({"question": "how do I use Anthropic?"})

'As Dario Amodei told me, Anthropic is a company that specializes in creating AI systems that can assist humans in various tasks. To use Anthropic\'s services, you should follow these steps:\\n\\n1. Visit the Anthropic website: Go to <https://www.anthropic.com/> to learn more about the company and its offerings.\\n2. Understand their products: Explore Anthropic\'s product suite to get a sense of the AI systems they offer. Their current offerings include COBRA, a chatbot designed for customer service, and CopyAI, a tool for AI-generated writing.\\n3. Assess your needs: Determine which Anthropic product best fits your requirements. Consider the scale of your operations, the complexity of your desired AI solutions, and your budget.\\n4. Reach out to Anthropic: Contact their team through the \\"Get in Touch\\" form on their website or via email at [hello@anthropic.com](mailto:hello@anthropic.com). They will be happy to help you navigate their products and answer any questions you may have.

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

"As Harrison Chase told me, LangChain is a platform that enables users to interact with multiple blockchain networks and cryptocurrencies. To use LangChain, follow these steps:\\n\\n1. Create an account: First, sign up for a LangChain account by visiting their official website. You'll need to provide your email address and create a password to register.\\n2. Understand the ecosystem: Before you start using LangChain, it's essential to familiarize yourself with the various blockchain networks and cryptocurrencies available on the platform. LangChain supports a wide range of blockchain networks, such as Bitcoin (BTC), Ethereum (ETH), and more.\\n3. Choose a wallet: LangChain provides multiple wallet options for storing and managing your cryptocurrencies. You can choose from hardware wallets, software wallets, or mobile wallets depending on your preferences and security requirements.\\n4. Fund your account: Once you have chosen a wallet, you need to fund your LangChain account with crypto

In [11]:
full_chain.invoke({"question": "whats 2 + 2"})

'Two plus two equals four.'